### Imports

In [1]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
path_py = "../ODE-RNN/Résultats/"
path_r = "../../data/synthetic_bph_1/Résultats random time/"
path_simul = "../../data/synthetic_bph_1/Simulations/"
CSV_Dtest = "../../data/synthetic_bph_1/Simulations/01_test.csv"

### Gathering results

In [3]:
pred_py = pd.DataFrame(columns=['individus', 'temps'])
for file in os.listdir(path_py):
    if "Prédictions" in file:
        df = pd.read_csv(path_py + file)
        pred_py = pd.merge(pred_py, df, on=['individus', 'temps'], how='outer')
pred_r_train = pd.read_csv(path_simul + 'simulation1.csv', sep=";", decimal=",")
pred_r_train = pred_r_train.loc[:,['individus', 'temps', 'y_fixed', 'y_fixed_obs', 'y_mixed', 'y_mixed_obs', "pred_mixed", "pred_fixed", "pred_naif_mixed", "pred_naif_fixed"]]
data_test =  pd.read_csv(CSV_Dtest, sep=";", decimal=",")
pred_r_test = pd.read_csv(path_r + 'Predictions.csv', sep=",", decimal=".")
pred_r_test = pred_r_test[['individus','temps', 'pred_mixed_1', 'pred_fixed_1', 'pred_naif_fixed_1', 'pred_naif_mixed_1']] #don't forget to add , 'pred_naif_mixed _1'

#### Network results

In [4]:
res_py = pd.DataFrame(columns=['Random effect', 'Model', "MAE moyenne sur l'entrainement", "MSE moyenne sur l'entrainement", "MAE moyenne sur l'entrainement bruité", "MSE moyenne sur l'entrainement bruité", "MAE moyenne sur le test", "MSE moyenne sur le test", "MAE moyenne sur le test bruité", "MSE moyenne sur le test bruité"])
for file in os.listdir(path_py):
    if "json" in file:
        df = pd.read_json(path_py + file)
        res_py = pd.concat((res_py,df))
res_py = res_py.sort_values(by=["MAE moyenne sur l'entrainement"])
res_py_fixed = res_py[res_py["Random effect"]=='Fixed'][['Model',"MAE moyenne sur l'entrainement", "MSE moyenne sur l'entrainement", "MAE moyenne sur l'entrainement bruité", "MSE moyenne sur l'entrainement bruité", "MAE moyenne sur le test", "MSE moyenne sur le test", "MAE moyenne sur le test bruité", "MSE moyenne sur le test bruité"]]
res_py_mixed = res_py[res_py["Random effect"]=='Mixed'][['Model',"MAE moyenne sur l'entrainement", "MSE moyenne sur l'entrainement", "MAE moyenne sur l'entrainement bruité", "MSE moyenne sur l'entrainement bruité", "MAE moyenne sur le test", "MSE moyenne sur le test", "MAE moyenne sur le test bruité", "MSE moyenne sur le test bruité"]]

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_13684\4058926855.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res_py = pd.concat((res_py,df))


#### Results of R models

In [5]:
res_r = pd.read_csv(path_r + "Résultats simulation.csv")

In [6]:
#res_r = pd.read_csv(path_r + "Performances_moyennes.csv")
res_oracle_mixed = res_r[[c for c in res_r.columns if ("naif" not in c) & ("lin" not in c) & ("mixed" in c) ]]
res_oracle_fixed = res_r[[c for c in res_r.columns if ("naif" not in c) & ("lin" not in c) & ("fixed" in c) ]]
res_naif_mixed = res_r[[c for c in res_r.columns if ("naif" in c) & ("mixed" in c)]]
res_naif_fixed = res_r[[c for c in res_r.columns if ("naif" in c) & ("fixed" in c)]]
res_lin_mixed = res_r[[c for c in res_r.columns if ("lin" in c) & ("mixed" in c)]]
res_lin_fixed = res_r[[c for c in res_r.columns if ("lin" in c) & ("fixed" in c)]]
res_oracle_mixed.rename(columns={'mae_train_mixed_truth':"MAE moyenne sur l'entrainement",
                                 'mse_train_mixed_truth':"MSE moyenne sur l'entrainement",
                                 'mae_test_mixed_truth':"MAE moyenne sur le test",
                                 'mse_test_mixed_truth':"MSE moyenne sur le test",
                                 "mae_train_mixed_obs": "MAE moyenne sur l'entrainement bruité",
                                 "mse_train_mixed_obs": "MSE moyenne sur l'entrainement bruité",
                                 "mae_test_mixed_obs": "MAE moyenne sur le test bruité",
                                 "mse_test_mixed_obs": "MSE moyenne sur le test bruité"
                                 }, inplace=True)
res_oracle_fixed.rename(columns={'mae_train_fixed_truth':"MAE moyenne sur l'entrainement",
                                 'mse_train_fixed_truth':"MSE moyenne sur l'entrainement",
                                 'mae_test_fixed_truth':"MAE moyenne sur le test",
                                 'mse_test_fixed_truth':"MSE moyenne sur le test",
                                 "mae_train_fixed_obs": "MAE moyenne sur l'entrainement bruité",
                                 "mse_train_fixed_obs": "MSE moyenne sur l'entrainement bruité",
                                 "mae_test_fixed_obs": "MAE moyenne sur le test bruité",
                                 "mse_test_fixed_obs": "MSE moyenne sur le test bruité"
                                 }, inplace=True)
res_naif_mixed.rename(columns={'mae_train_naif_mixed_truth':"MAE moyenne sur l'entrainement",
                                 'mse_train_naif_mixed_truth':"MSE moyenne sur l'entrainement",
                                 'mae_test_naif_mixed_truth':"MAE moyenne sur le test",
                                 'mse_test_naif_mixed_truth':"MSE moyenne sur le test",
                                 "mae_train_naif_mixed_obs": "MAE moyenne sur l'entrainement bruité",
                                 "mse_train_naif_mixed_obs": "MSE moyenne sur l'entrainement bruité",
                                 "mae_test_naif_mixed_obs": "MAE moyenne sur le test bruité",
                                 "mse_test_naif_mixed_obs": "MSE moyenne sur le test bruité"
                                 }, inplace=True)
res_naif_fixed.rename(columns={'mae_train_naif_fixed_truth':"MAE moyenne sur l'entrainement",
                                 'mse_train_naif_fixed_truth':"MSE moyenne sur l'entrainement",
                                 'mae_test_naif_fixed_truth':"MAE moyenne sur le test",
                                 'mse_test_naif_fixed_truth':"MSE moyenne sur le test",
                                 "mae_train_naif_fixed_obs": "MAE moyenne sur l'entrainement bruité",
                                 "mse_train_naif_fixed_obs": "MSE moyenne sur l'entrainement bruité",
                                 "mae_test_naif_fixed_obs": "MAE moyenne sur le test bruité",
                                 "mse_test_naif_fixed_obs": "MSE moyenne sur le test bruité"
                                 }, inplace=True)
res_lin_mixed.rename(columns={'mae_train_lin_mixed_truth':"MAE moyenne sur l'entrainement",
                        'mse_train_lin_mixed_truth':"MSE moyenne sur l'entrainement", 
                        'mae_test_lin_mixed_truth':"MAE moyenne sur le test",
                        'mse_test_lin_mixed_truth':"MSE moyenne sur le test",
                        'mae_train_lin_mixed_obs':"MAE moyenne sur l'entrainement bruité",  
                        'mse_train_lin_mixed_obs':"MSE moyenne sur l'entrainement bruité", 
                        'mae_test_lin_mixed_obs':"MAE moyenne sur le test bruité",
                        'mse_test_lin_mixed_obs':"MSE moyenne sur le test bruité"
                        }, inplace=True)
res_lin_fixed.rename(columns={'mae_train_lin_fixed_truth':"MAE moyenne sur l'entrainement",
                        'mse_train_lin_fixed_truth':"MSE moyenne sur l'entrainement", 
                        'mae_test_lin_fixed_truth':"MAE moyenne sur le test",
                        'mse_test_lin_fixed_truth':"MSE moyenne sur le test",
                        'mae_train_lin_fixed_obs':"MAE moyenne sur l'entrainement bruité",  
                        'mse_train_lin_fixed_obs':"MSE moyenne sur l'entrainement bruité", 
                        'mae_test_lin_fixed_obs':"MAE moyenne sur le test bruité",
                        'mse_test_lin_fixed_obs':"MSE moyenne sur le test bruité"
                        }, inplace=True)
res_oracle_mixed.loc[:,['Model']] = 'Oracle'
res_oracle_fixed.loc[:,['Model']] = 'Oracle'
res_naif_mixed.loc[:,['Model']] = 'Naif'
res_naif_fixed.loc[:,['Model']] = 'Naif'
res_lin_mixed.loc[:,['Model']] = 'Linéaire'
res_lin_fixed.loc[:,['Model']] = 'Linéaire'

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_13684\541684509.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_oracle_mixed.rename(columns={'mae_train_mixed_truth':"MAE moyenne sur l'entrainement",
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_13684\541684509.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_oracle_fixed.rename(columns={'mae_train_fixed_truth':"MAE moyenne sur l'entrainement",
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_13684\541684509.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

In [7]:
res_fixed = pd.concat((res_py_fixed, res_oracle_fixed, res_naif_fixed, res_lin_fixed))
res_mixed = pd.concat((res_py_mixed, res_oracle_mixed, res_naif_mixed, res_lin_mixed))

## Performances in predicting with fixed effects

In [8]:
res_fixed = res_fixed.groupby('Model').mean()

In [9]:
res_fixed

,MAE moyenne sur l'entrainement,MSE moyenne sur l'entrainement,MAE moyenne sur l'entrainement bruité,MSE moyenne sur l'entrainement bruité,MAE moyenne sur le test,MSE moyenne sur le test,MAE moyenne sur le test bruité,MSE moyenne sur le test bruité
Model,,,,,,,,
Linéaire,0.270319,0.131749,0.759566,0.911257,0.268646,0.130695,0.753783,0.898520
Naif,1.091802,3.014597,1.358219,3.865203,1.043257,2.856071,1.319454,3.702566
ODE RNN,0.554900,0.627504,0.859089,1.197236,1.128135,3.887676,1.482106,4.865125
Oracle,0.013921,0.000330,0.798325,1.001108,0.013620,0.000313,0.790672,0.985670
RNN,0.655433,0.882463,0.932276,1.456888,1.172266,4.232730,1.504819,5.209120


### graphs for training

In [ ]:
n = 5
ex = np.random.choice(pd.unique(pred_py['individus']), n)
fig, axs = plt.subplots(n)
fig.set_figwidth(20)
fig.set_figheight(50)
for k in range(n):
    axs[k].plot(pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['temps'],
                pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['y_fixed_obs'], 
                label= 'target w/ noise')
    axs[k].plot(pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['temps'],
                pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['y_fixed'], 
                label= 'target')
    axs[k].plot(pred_py[(pred_py['temps']!=0) & (pred_py['individus']== ex[k])]['temps'],
                pred_py[(pred_py['temps']!=0) & (pred_py['individus']==ex[k])]['y_pred_RNN_Fixed'], 
                label= 'prediction RNN', linestyle='dashdot')
    axs[k].plot(pred_py[(pred_py['temps']!=0) & (pred_py['individus']== ex[k])]['temps'],
                pred_py[(pred_py['temps']!=0) & (pred_py['individus']==ex[k])]['y_pred_ODE RNN_Fixed'], 
                label= 'prediction ODE RNN', linestyle='dashdot')
    axs[k].plot(pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['temps'],
                pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['pred_naif_fixed'], 
                label= 'prediction naïve', linestyle='dotted')
    axs[k].axvline(x = 5, color = 'gray', linestyle='dotted')
    axs[k].legend()
plt.savefig("../../models/ODE-RNN/Résultats/Graphs/Train_fixed.png")

### graphs for test

In [ ]:
n = 5
ex = np.random.choice(pd.unique(pred_py['individus']), n)
fig, axs = plt.subplots(n)
fig.set_figwidth(20)
fig.set_figheight(50)
for k in range(n):
    axs[k].plot(data_test[(pred_r_test['temps']!=0) & (data_test['individus']== ex[k])]['temps'],
                data_test[(data_test['temps']!=0) & (data_test['individus']== ex[k])]['y_fixed_obs'], 
                label= 'target w/ noise')
    axs[k].plot(data_test[(data_test['temps']!=0) & (data_test['individus']== ex[k])]['temps'],
                data_test[(data_test['temps']!=0) & (data_test['individus']== ex[k])]['y_fixed'], 
                label= 'target')
    axs[k].plot(pred_py[(pred_py['temps']!=0) & (pred_py['individus']== ex[k])]['temps'],
                pred_py[(pred_py['temps']!=0) & (pred_py['individus']==ex[k])]['y_test_RNN_Fixed'], 
                label= 'prediction RNN', linestyle='dashdot')
    axs[k].plot(pred_py[(pred_py['temps']!=0) & (pred_py['individus']== ex[k])]['temps'],
                pred_py[(pred_py['temps']!=0) & (pred_py['individus']==ex[k])]['y_test_ODE RNN_Fixed'], 
                label= 'prediction ODE RNN', linestyle='dashdot')
    axs[k].plot(pred_r_test[(pred_r_test['temps']!=0) & (pred_r_test['individus']== ex[k])]['temps'],
                pred_r_test[(pred_r_test['temps']!=0) & (pred_r_test['individus']== ex[k])]['pred_naif_fixed_1'], 
                label= 'prediction naïve', linestyle='dotted')
    """axs[k].plot(pred_r_test[(pred_r_test['temps']!=0) & (pred_r_test['individus']== ex[k])]['temps'],
                pred_r_test[(pred_r_test['temps']!=0) & (pred_r_test['individus']== ex[k])]['pred_fixed_1'], 
                label= 'prediction oracle', linestyle='dotted')"""
    axs[k].axvline(x = 5, color = 'gray', linestyle='dotted')
    axs[k].legend()
plt.savefig("../../models/ODE-RNN/Résultats/Graphs/Test_fixed.png")

## Performance in predicting with mixed effects

In [ ]:
res_mixed[[c for c in res_mixed.columns if '_' not in c]]

### graphs for training

In [ ]:
n = 5
ex = np.random.choice(pd.unique(pred_py['individus']), n)
fig, axs = plt.subplots(n)
fig.set_figwidth(20)
fig.set_figheight(50)
for k in range(n):
    axs[k].plot(pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['temps'],
                pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['y_mixed_obs'], 
                label= 'target w/ noise')
    axs[k].plot(pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['temps'],
                pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['y_mixed'], 
                label= 'target')
    axs[k].plot(pred_py[(pred_py['temps']!=0) & (pred_py['individus']== ex[k])]['temps'],
                pred_py[(pred_py['temps']!=0) & (pred_py['individus']==ex[k])]['y_pred_RNN_Mixed'], 
                label= 'prediction RNN', linestyle='dashdot')
    axs[k].plot(pred_py[(pred_py['temps']!=0) & (pred_py['individus']== ex[k])]['temps'],
                pred_py[(pred_py['temps']!=0) & (pred_py['individus']==ex[k])]['y_pred_ODE RNN_Mixed'], 
                label= 'prediction ODE RNN', linestyle='dashdot')
    axs[k].plot(pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['temps'],
                pred_r_train[(pred_r_train['temps']!=0) & (pred_r_train['individus']== ex[k])]['pred_naif_mixed'], 
                label= 'prediction naïve', linestyle='dotted')
    axs[k].axvline(x = 5, color = 'gray', linestyle='dotted')
    axs[k].legend()
plt.savefig("../../models/ODE-RNN/Résultats/Graphs/Train_mixed.png")

### graphs for testing

In [ ]:
n = 5
ex = np.random.choice(pd.unique(pred_py['individus']), n)
fig, axs = plt.subplots(n)
fig.set_figwidth(20)
fig.set_figheight(50)
for k in range(n):
    axs[k].plot(data_test[(pred_r_train['temps']!=0) & (data_test['individus']== ex[k])]['temps'],
                data_test[(data_test['temps']!=0) & (data_test['individus']== ex[k])]['y_mixed_obs'], 
                label= 'target w/ noise')
    axs[k].plot(data_test[(data_test['temps']!=0) & (data_test['individus']== ex[k])]['temps'],
                data_test[(data_test['temps']!=0) & (data_test['individus']== ex[k])]['y_mixed'], 
                label= 'target')
    axs[k].plot(pred_py[(pred_py['temps']!=0) & (pred_py['individus']== ex[k])]['temps'],
                pred_py[(pred_py['temps']!=0) & (pred_py['individus']==ex[k])]['y_test_RNN_Mixed'], 
                label= 'prediction RNN', linestyle='dashdot')
    axs[k].plot(pred_py[(pred_py['temps']!=0) & (pred_py['individus']== ex[k])]['temps'],
                pred_py[(pred_py['temps']!=0) & (pred_py['individus']==ex[k])]['y_test_ODE RNN_Mixed'], 
                label= 'prediction ODE RNN', linestyle='dashdot')
    axs[k].plot(pred_r_test[(pred_r_test['temps']!=0) & (pred_r_test['individus']== ex[k])]['temps'],
                pred_r_test[(pred_r_test['temps']!=0) & (pred_r_test['individus']== ex[k])]['pred_naif_mixed_1'], 
                label= 'prediction naïve', linestyle='dotted')
    axs[k].axvline(x = 5, color = 'gray', linestyle='dotted')
    axs[k].legend()
plt.savefig("../../models/ODE-RNN/Résultats/Graphs/Test_mixed.png")